# Revised case normalization for Hirslanden Salem 2018

This jupyter notebook is used to normalize the revised case from DtoD.

Before runing the notebook, raw_data folder need to added to the root directory

The raw data folder can be find here: https://aimedic.sharepoint.com/:f:/s/dev/Ejx_A1dg8gtPumFknOWOh0oBi6ofx9hctYiq3c-0gH9vYA?e=UmcgrS

Normalization:

-  Convert the column names to the name used in the Database
-  Delete cases which is empty in the follow columns VALIDATION_COLS: 'case_id', 'patient_id', 'gender', 'age_years', duration_of_stay',  'pccl',  'drg'
- choose neccessary columns COLS_TO_SELECT: case_id, patient_id, gender, age_years, duration_of_stay, pccl, drg, pd, bfs_code, added_icds, removed_icds, added_chops, removed_chops
- still need to do (TODO):    
    -  Check CHOP upper/lowercase
    -  Check whether the PD changed. If it did, new and old PD are stored together with added and removed ICDs, respectively
    -  Pad case IDs with 0s
    -  Write function to validate cases


In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018'])

In [3]:
file = FILES_TO_ANALYZE['Hirslanden Salem 2018']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2018', sheets=['Änderungen _SA_2018'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 46 cases for Hirslanden Salem 2018
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,0041602986,90C3E16484AD653A,M,88,12,3,I43B,M171,M200,N183,<NA>,<NA>,<NA>
1,0041637726,D774DF60034ACAA4,W,53,19,3,I09C,T842,M200,"K564,T811",<NA>,<NA>,<NA>
2,0041641561,4A03EA9845D13800,M,81,9,3,L60C,N185,M100,N1793,<NA>,<NA>,<NA>
3,0041643625,E3CD9E9C233B926F,M,83,10,3,F13C,E1172,M100,M8617,M8697,<NA>,<NA>
4,0041644263,80BD3429C23B3A1E,W,76,10,3,T60E,A4152,M200,"A418,J9600,B962","A4152,J9699",<NA>,<NA>


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['0041602986', '0041637726', '0041641561', '0041643625',
       '0041644263', '0041649868', '0041656883', '0041683635',
       '0041689417', '0041695908', '0041702122', '0041710615',
       '0041711681', '0041715963', '0041735592', '0041760287',
       '0041763118', '0041776339', '0041779228', '0041779882',
       '0041788723', '0041800898', '0041812167', '0041813832',
       '0041821441', '0041869626', '0041870623', '0041638807',
       '0041916526', '0041918398', '0041630555', '0041881668',
       '0041898735', '0041675045', '0041691601', '0041742275',
       '0041785404', '0041789235', '0041679610', '0041823802',
       '0041826915', '0041830184', '0041837846', '0041850795',
       '0041869375', '0041927781'], dtype=object)

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 0 out of 46 revised cases from DtoD that are matched with the database for Hirslanden Salem 2018


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match